# CartPole AI

### Getting Set up

To run locally you need the following software installed:

* Anaconda
* AI Gym
* PyTorch

You can get Anaconda from [here](https://www.anaconda.com/download/).

Once Anaconda is installed, you'll need to install PyTorch.  You can do so by going to the [PyTorch website](https://pytorch.org) and following the instructions appropriate to what you're trying to install.

**PyTorch tl;dr for Mac**: run `conda install pytorch-cpu torchvision-cpu -c pytorch`

After installing PyTorch you'll need Gym.  Unfortunately there's no conda package for Gym, so it's a few extra steps.

* run `conda install libgcc`
* run `pip install gym`
* run `pip install box2d-py`

To create animations you'll need install the following:

* run `pip install git+https://github.com/jakevdp/JSAnimation.git`
* run `pip install pyglet==1.2.4` (this will downgrade pyglet to work with JSAnimation)

**For Mac**
* run `brew install ffmpeg`

**For Debian Based Linux/Ubuntu**
* run `apt install ffmpeg`

Once you're all done, you can go to the directory where you cloned the project and type:

`jupyter notebook`

If Anaconda is installed properly this should run the Jupyter notebook server.  Open a browser window and navigate to `localhost:8888`.  Once there, click on the `Session 4.ipynb` link.

To execute the code cells below, click in the cell and hit **ctrl/cmd + enter**

In [1]:
import setup
from imports import *
from buffer import Transition, MemoryRing

In [2]:
setup.check_my_system()

You have a CUDA capable machine
Available GPU(s):
	cuda:0 TITAN V (CUDA version 7.0)
	cuda:1 GeForce GTX 1050 Ti (CUDA version 6.1)

Your AI Gym enviornment is correctly set up.


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.inputLayer = nn.Linear(4, 128)
        self.drop1 = nn.Dropout(0.2)
        self.hiddenLayer = nn.Linear(128, 64)
        self.outputLayer = nn.Linear(64, 2)
        
    def forward(self, state):
        x = F.relu(self.inputLayer(state))
        x = self.drop1(x)
        x = F.relu(self.hiddenLayer(x))
        x = F.relu(self.outputLayer(x))
        return x

In [ ]:
class Agent(object):
    def __init__(self, lr, action_space_size, eps_max = 0.9, eps_min = 0.05, esp_decay = 0.998):
        self.learning_rate = lr
        self.epsilon_max = eps_max
        self.epsilon_min = eps_min
        self.epsilon_decay = eps_decay
        self.action_space_size = action_space_size
        
        self.memory = MemoryRing(2**16)
        self.decay_timer = 0
        self.gamma = 0.999
        self.batch_size = 32
        self.step = 0
        
        self.policy_net = DQN().to(device)
        self.target_net = DQN().to(device)
        
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.target_net_update = 50
        
        self.loss = torch.nn.MSELoss()
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)